In [1]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

OUTPUT_DIR = "../data/model_output"

folder_paths = [OUTPUT_DIR]

# Create the output directories if they doesn't exist
for folder_path in folder_paths:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [2]:
import albumentations as A
from plot import get_bounding_boxes, create_bboxes_image
import cv2
import numpy as np
from layers import AMF_GD_YOLOv8
import matplotlib.pyplot as plt
from time import time
import random
import torch

In [3]:
print(torch.cuda.is_available())

True


In [4]:
from ultralytics import YOLO
from ultralytics.models.yolo.detect.predict import DetectionPredictor

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolov8n.pt")

# Perform object detection on an image using the model
results = model("https://ultralytics.com/images/bus.jpg")

print(results[0].boxes)


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
y.shape = torch.Size([1, 84, 8400])
y[0] = tensor([[3.4759e+00, 2.6257e+01, 2.6180e+01,  ..., 5.3251e+02, 5.5289e+02, 5.7816e+02],
        [6.4561e+00, 4.6107e+00, 3.7780e+00,  ..., 5.9229e+02, 5.9316e+02, 5.8190e+02],
        [7.0762e+00, 5.1426e+01, 4.9170e+01,  ..., 2.2848e+02, 2.0270e+02, 1.3878e+02],
        ...,
        [5.8455e-07, 4.6036e-07, 4.5342e-07,  ..., 1.3136e-06, 1.0954e-06, 1.9521e-06],
        [2.1589e-07, 2.5406e-07, 2.0488e-07,  ..., 1.1909e-06, 1.1038e-06, 1.2457e-06],
        [1.9636e-07, 1.3884e-07, 1.4594e-07,  ..., 1.5077e-06, 1.3969e-06, 1.5349e-06]], device='cuda:0')
y.shape = torch.Size([1, 84, 6300])
y[0] = tensor([[3.5238e+00, 2.3263e+01, 2.9372e+01,  ..., 3.7811e+02, 4.0109e+02, 4.5012e+02],
        [7.3379e+00, 3.9321e+00, 3.4007e+00,  ..., 6.1498e+02, 5.8362e+02, 5.7263e+02],
        [7.3691e+00, 4.6660e+01, 5.8205e+01,  ..., 2.1778e+02, 2.1653e+02, 8.4504e+01],
        ...,
        [2.35

### Use the AMF GD YOLOv8 model

In [5]:
class_names = {0: "Tree", 1: "Tree_disappeared", 2: "Tree_replaced", 3: "Tree_new"}
class_indices = {value: key for key, value in class_names.items()}

model = AMF_GD_YOLOv8(3, 1, scale="s", class_names=class_names)

rgb_image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_0_1.tif"
chm_image_path = (
    "../data/CHM_cropped_0p24/122000_484000/122000_484000_0_1_filtered_chm.tif"
)
annotations_path = "../data/annotations_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_0_1.json"

bboxes, classes = get_bounding_boxes(annotations_path)

output_name = "Model_output_test.jpg"
output_path = os.path.join(OUTPUT_DIR, output_name)

output = model(rgb_image_path, chm_image_path, image_save_path=output_path)

y.shape = torch.Size([1, 8, 140])
y[0] = tensor([[-0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.6577, 0.4613, 0.5746,  ..., 0.5281, 0.5232, 0.5120],
        [0.2921, 0.1740, 0.1406,  ..., 0.5486, 0.4542, 0.5081],
        [0.2966, 0.6424, 0.3509,  ..., 0.4466, 0.4006, 0.5065]], grad_fn=<SelectBackward0>)
xi = 0
i = tensor([  7,  80,  83,   5,  84, 122, 119,  81,   4,  91,  90,   8,   0,  43,   1,   3,  76,  14, 104, 124,  94, 117,  86, 111,  50, 123,  23,  41,  97,   9, 106,  40,  77,  10, 133,  57,  35,  39, 112,  15, 125, 113,  42,  61,  52,  49,  53, 115,  58,  88, 121, 128,  85,  11,  16, 100,  46,  13,  45,  92, 131, 102,
         56,   2,  82, 103, 105, 118,  12, 101,  55,  30,  31,  51,  68, 109,   6,  47,  19,  34,  67, 110,  64,  32,  44,  59, 116,  95, 120,  29,  87,  54,  33,  99, 130, 134,  26,  38,  69, 108,  79,  

In [34]:
from ultralytics.models.utils.loss import DETRLoss


def xywh2xyxy(x):
    """
    Convert bounding box coordinates from (x, y, width, height) format to (x1, y1, x2, y2) format where (x1, y1) is the
    top-left corner and (x2, y2) is the bottom-right corner.

    Args:
        x (np.ndarray | torch.Tensor): The input bounding box coordinates in (x, y, width, height) format.

    Returns:
        y (np.ndarray | torch.Tensor): The bounding box coordinates in (x1, y1, x2, y2) format.
    """
    assert (
        x.shape[-1] == 4
    ), f"input shape last dimension expected 4 but input shape is {x.shape}"
    y = (
        torch.empty_like(x) if isinstance(x, torch.Tensor) else np.empty_like(x)
    )  # faster than clone/copy
    w = x[..., 2]  # half-width
    h = x[..., 3]  # half-height
    y[..., 0] = x[..., 0]  # top left x
    y[..., 1] = x[..., 1]  # top left y
    y[..., 2] = x[..., 0] + w  # bottom right x
    y[..., 3] = x[..., 1] + h  # bottom right y
    return y


nc = 4
loss_func = DETRLoss(nc=nc)

# output[0][0][0, :, 0] = torch.Tensor([107.3209, 151.2296, 161.4145, 207.6499, 0.9837, 0.5472, 0.3150, 0.5183])

pred_bboxes = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, :4]
pred_scores = output[0][0].unsqueeze(0).permute((0, 1, 3, 2))[:, :, :, 4:]
print(pred_scores)
batch = {
    "cls": torch.Tensor([class_indices[cls] for cls in classes]).to(dtype=torch.int64),
    "bboxes": xywh2xyxy(torch.Tensor(bboxes)),
    "gt_groups": [len(classes)],
}

print(f"{pred_bboxes.shape = }")
print(f"{pred_scores.shape = }")
print(f"{batch["cls"].shape = }")
print(f"{batch["bboxes"].shape = }")
print(f"{len(batch["gt_groups"]) = }")

print(f"{batch["bboxes"][0] = }")
print(f"{batch["cls"][0] = }")

prefect_pred_bboxes = batch["bboxes"].unsqueeze(0).unsqueeze(0)
prefect_pred_scores = (
    torch.where(
        batch["cls"].unsqueeze(-1).repeat(1, nc)
        == torch.arange(nc).unsqueeze(0).repeat(batch["cls"].shape[0], 1),
        1.0,
        0.0,
    )
    .unsqueeze(0)
    .unsqueeze(0)
)

loss = loss_func.forward(prefect_pred_bboxes, prefect_pred_scores, batch)
print(f"{loss = }")

print(f"{output[0][0].shape = }")
print(f"{output[0][1][0].shape = }")
print(f"{output[0][1][1].shape = }")
print(f"{output[0][1][2].shape = }")
# print(f"{output[1].boxes = }")
print(f"{output[0][0][0, :, 0] = }")

tensor([[[[0.9837, 0.5472, 0.3150, 0.5183],
          [0.3291, 0.4613, 0.1740, 0.6424],
          [0.1906, 0.5746, 0.1406, 0.3509],
          [0.1456, 0.6352, 0.3711, 0.3502],
          [0.1612, 0.5967, 0.6915, 0.2346],
          [0.7133, 0.5498, 0.5533, 0.2348],
          [0.3900, 0.4772, 0.5607, 0.1314],
          [0.3373, 0.7607, 0.5246, 0.5656],
          [0.2355, 0.6602, 0.5475, 0.4714],
          [0.3644, 0.6163, 0.5072, 0.4572],
          [0.3429, 0.6107, 0.3755, 0.4262],
          [0.4036, 0.5824, 0.5181, 0.4684],
          [0.3764, 0.5703, 0.5322, 0.4865],
          [0.4151, 0.5789, 0.5407, 0.5235],
          [0.4281, 0.5465, 0.6327, 0.5742],
          [0.4815, 0.6021, 0.5794, 0.5128],
          [0.4506, 0.5663, 0.5812, 0.4468],
          [0.4058, 0.4587, 0.5249, 0.4422],
          [0.3443, 0.5080, 0.5293, 0.5430],
          [0.3881, 0.5596, 0.4437, 0.5264],
          [0.4292, 0.5266, 0.4753, 0.4964],
          [0.5238, 0.5270, 0.4896, 0.4875],
          [0.3727, 0.4369, 0.503

In [29]:
t = batch["cls"]

# Use torch.where to conditionally set values to 0.0
result = torch.where(
    t.unsqueeze(-1).repeat(1, 4) == torch.arange(4).unsqueeze(0).repeat(101, 1),
    1.0,
    0.0,
)

print(result)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1.,

### How to use data augmentation with Albumentations

In [4]:
from typing import Sequence, Dict, Any


class Pad(A.PadIfNeeded):
    def __init__(
        self,
        added_height: int | None = 64,
        added_width: int | None = 64,
        pad_height_divisor: int | None = None,
        pad_width_divisor: int | None = None,
        position: A.PadIfNeeded.PositionType | str = A.PadIfNeeded.PositionType.CENTER,
        border_mode: int = cv2.BORDER_REFLECT_101,
        value: float | Sequence[float] | None = None,
        mask_value: float | Sequence[float] | None = None,
        always_apply: bool = False,
        p: float = 1,
    ):
        super().__init__(
            0,
            0,
            pad_height_divisor,
            pad_width_divisor,
            position,
            border_mode,
            value,
            mask_value,
            always_apply,
            p,
        )
        self.added_height = added_height
        self.added_width = added_width

    def update_params(self, params: Dict[str, Any], **kwargs: Any) -> Dict[str, Any]:
        params = super().update_params(params, **kwargs)
        rows = params["rows"]
        cols = params["cols"]
        self.min_height = rows + self.added_height
        self.min_width = cols + self.added_width
        return super().update_params(params, **kwargs)

In [222]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

bbox_params = A.BboxParams(
    format="coco", min_area=0, min_visibility=0.2, label_fields=["class_labels"]
)

transform_spatial = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.GridDistortion(
            num_steps=distort_steps,
            distort_limit=(-distort_limit, distort_limit),
            border_mode=cv2.BORDER_CONSTANT,
            normalized=True,
            p=0.5,
        ),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=1.0),
        A.Perspective(interpolation=cv2.INTER_LINEAR, p=0.25),
    ],
    bbox_params=bbox_params,
)

transform_pixel = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        # A.Normalize(p=1.0),
    ],
)

# image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.tif"
# annotations_path = "../data/annotations_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.json"
image_path = "../data/images_full/2023_122000_484000_RGB_hrl.tif"
annotations_path = "../data/annotations_full/3_all_annotations.json"

t = time()

image = cv2.imread(image_path)
bboxes, labels = get_bounding_boxes(annotations_path)

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 10

colors = {
    "Tree": (104, 201, 45),
    "Tree_unsure": (255, 215, 158),
    "Tree_disappeared": (158, 174, 255),
    "Tree_replaced": (255, 90, 82),
    "Tree_new": (251, 106, 225),
}

t = time()

for i in range(number_tests):
    transformed_spatial = transform_spatial(
        image=image, bboxes=bboxes, class_labels=labels
    )
    transformed_spatial_image = transformed_spatial["image"]
    transformed_bboxes = transformed_spatial["bboxes"]
    transformed_class_labels = transformed_spatial["class_labels"]
    transformed = transform_pixel(image=transformed_spatial_image)
    transformed_image = transformed["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image)

    bboxes_image = create_bboxes_image(
        transformed_image,
        transformed_bboxes,
        labels=transformed_class_labels,
        colors_dict=colors,
        scores=np.random.rand((len(bboxes))),
    )

    output_name = f"Augmentation_test_{i}_bboxes.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, bboxes_image)

print(f"Time elapsed: {round(time() - t, 3)}")

Time elapsed: 1.737
Time elapsed: 1.257


In [212]:
crop_size = 640
distort_steps = 30
distort_limit = 0.2

transform = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
    ],
)

transform_2 = A.Compose(
    [
        A.RandomCrop(width=crop_size, height=crop_size, p=1.0),
        A.Sharpen(p=0.25),
        A.RingingOvershoot(p=0.5),
        A.RandomGamma(p=1.0),
        A.GaussianBlur(p=0.5),
        A.GaussNoise(p=0.5),
        A.FancyPCA(alpha=1.0, p=0.5),
        A.Emboss(p=0.5),
        A.Blur(p=0.5),
        A.RandomBrightnessContrast(p=1.0),
        A.ChannelDropout(p=0.25),
        A.CLAHE(clip_limit=2.0, p=0.25),
        A.Normalize(p=1.0),
    ],
)

image_path = "../data/images_cropped/2023_122000_484000_RGB_hrl/2023_122000_484000_RGB_hrl_2_3.tif"
# image_path = "../data/flat-design-abstract-outline-background_23-2150616456.jpg"

t = time()

image = cv2.imread(image_path)

print(f"Time elapsed: {round(time() - t, 3)}")

number_tests = 1

t = time()

for i in range(number_tests):
    r = random.random()
    random.seed(r)
    transformed_image = transform(image=image)["image"]
    random.seed(r)
    transformed_image_2 = transform_2(image=image)["image"]

    output_name = f"Augmentation_test_{i}.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image)

    output_name = f"Augmentation_test_{i}_2.tif"
    output_path = os.path.join(OUTPUT_DIR, output_name)
    cv2.imwrite(output_path, transformed_image_2)

print(f"Time elapsed: {round(time() - t, 3)}")

Time elapsed: 0.014
Time elapsed: 0.061
